In [2]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

print(os.getcwd())

/Users/sumitsingh/Documents/GitHub/mlproject/StockAppFrontend/ml_model


In [3]:
# Load Data
data = pd.read_csv(
    "/Users/sumitsingh/Documents/GitHub/mlproject/StockAppFrontend/ml_model/source/indexProcessed.csv"
)
prices = data["Close"].values

# Normalize Data
scaler = MinMaxScaler()
prices_scaled = scaler.fit_transform(prices.reshape(-1, 1))

In [4]:
# Prepare Features
window_size = 60  # Number of past days to use as input
X, y = [], []
for i in range(window_size, len(prices_scaled)):
    X.append(prices_scaled[i - window_size : i, 0])
    y.append(prices_scaled[i, 0])

X, y = np.array(X), np.array(y)
X = X.reshape((X.shape[0], X.shape[1], 1))  # Reshape for LSTM

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [5]:
# Build LSTM Model
model = tf.keras.Sequential(
    [
        tf.keras.layers.LSTM(50, return_sequences=True, input_shape=(X.shape[1], 1)),
        tf.keras.layers.LSTM(50, return_sequences=False),
        tf.keras.layers.Dense(25),
        tf.keras.layers.Dense(1),
    ]
)

model.compile(optimizer="adam", loss="mean_squared_error")
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Save the Model
model.save("stock_prediction_model.h5")

# Predict
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)

Epoch 1/10


/Users/sumitsingh/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2605/2605 ━━━━━━━━━━━━━━━━━━━━ 85s 32ms/step - loss: 5.0847e-04 - val_loss: 2.1839e-05
Epoch 2/10
2605/2605 ━━━━━━━━━━━━━━━━━━━━ 59s 23ms/step - loss: 3.4478e-05 - val_loss: 2.6212e-05
Epoch 3/10
2605/2605 ━━━━━━━━━━━━━━━━━━━━ 650s 250ms/step - loss: 1.8977e-05 - val_loss: 1.2905e-05
Epoch 4/10
2605/2605 ━━━━━━━━━━━━━━━━━━━━ 48s 19ms/step - loss: 2.0277e-05 - val_loss: 1.6820e-05
Epoch 5/10
2605/2605 ━━━━━━━━━━━━━━━━━━━━ 769s 295ms/step - loss: 2.0727e-05 - val_loss: 9.1241e-06
Epoch 6/10
2605/2605 ━━━━━━━━━━━━━━━━━━━━ 50s 19ms/step - loss: 1.6861e-05 - val_loss: 1.1414e-05
Epoch 7/10
2605/2605 ━━━━━━━━━━━━━━━━━━━━ 49s 19ms/step - loss: 1.5908e-05 - val_loss: 1.9426e-04
Epoch 8/10
2605/2605 ━━━━━━━━━━━━━━━━━━━━ 45s 17ms/step - loss: 1.7112e-05 - val_loss: 9.1649e-06
Epoch 9/10
2605/2605 ━━━━━━━━━━━━━━━━━━━━ 46s 17ms/step - loss: 1.6410e-05 - val_loss: 9.7670e-06
Epoch 10/10
2605/2605 ━━━━━━━━━━━━━━━━━━━━ 47s 18ms/step - loss: 1.2751e-05 - val_loss: 1.3586e-05


652/652 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step


In [6]:
from tf.keras.models import load_model

model = load_model("stock_prediction_model.h5")

model.summary()

ModuleNotFoundError: No module named 'tf'

In [11]:
model_json = model.to_json()

with open("model_architecture.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
# Install the required package
%pip install tensorflow

from tensorflow.keras.utils import plot_model

plot_model(
    model, to_file="model_architecture.png", show_shapes=True, show_layer_names=True
)